# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
from pprint import pprint

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
output_data_file = "../output_data/cities.csv"
# output_data_file

types_df = pd.read_csv(output_data_file)

weather_data_df = types_df.drop(columns=["Unnamed: 0"])
weather_data_df


,City,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude
0,Chui,58.53,73,69,15.46,-33.70,-53.46
1,Puerto Ayora,71.91,69,20,13.87,-0.74,-90.35
2,Albany,67.21,82,22,5.01,42.60,-73.97
3,Suntar,13.84,96,56,3.76,62.14,117.63
4,Lebu,50.05,77,0,15.73,-37.62,-73.65
...,...,...,...,...,...,...,...
546,Cidreira,66.38,89,100,15.59,-30.18,-50.21
547,Jamestown,56.86,93,90,4.70,42.10,-79.24
548,Ushuaia,53.60,46,75,12.75,-54.80,-68.30
549,Blama,73.62,96,83,1.14,7.87,-11.34


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

# Store Humidity in humidity
humidity = weather_data_df["Humidity (%)"]

In [9]:

# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data_df.loc[(weather_data_df["Wind Speed (mph)"] <= 10) & (weather_data_df["Cloud Coverage (%)"] == 0) & \
                                   (weather_data_df["Temperature (F)"] >= 70) & (weather_data_df["Temperature (F)"] <= 80)].dropna()

narrowed_city_df

,City,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude
60,Minna,77.23,64,0,1.90,9.61,6.56
90,Chabahar,75.63,44,0,3.42,25.29,60.64
499,Dutse,76.30,30,0,3.02,11.76,9.34


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [39]:
for index, row in narrowed_city_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    city_name = row["City"]
    
params = {
        "type" : "hotel",
        "keyword" : "hotel",
        "radius" : 5000,
        "key" : g_key}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
requested = requests.get(base_url, params=params)
jsn = requested.json()
try:
        hotellist.append(jsn['results'][0]['name'])
except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude,Hotel Name
60,Minna,77.23,64,0,1.90,9.61,6.56,
90,Chabahar,75.63,44,0,3.42,25.29,60.64,
499,Dutse,76.30,30,0,3.02,11.76,9.34,


In [40]:
# Create a hotel_df
hotel_df = narrowed_city_df

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Temperature (F),Humidity (%),Cloud Coverage (%),Wind Speed (mph),Latitude,Longitude,Hotel Name
60,Minna,77.23,64,0,1.90,9.61,6.56,
90,Chabahar,75.63,44,0,3.42,25.29,60.64,
499,Dutse,76.30,30,0,3.02,11.76,9.34,


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

KeyError: 'Country'

In [ ]:

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig